In [125]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [126]:
# url of 23/24 PL games
url = 'https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures'

In [ ]:
# get the table of fixtures
df = pd.read_html(url, 
                 attrs={"id":"sched_2023-2024_9_1"}
                 )

In [127]:
# Initializing parser
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")


## First exploration

In [ ]:
# Get the html pf the fictures table 
table = soup.find(id='sched_2023-2024_9_1').find("tbody")

In [ ]:
table.find("tr")

In [ ]:
table.find("tr").find('td', class_='right', attrs={'data-stat': 'home_team'}).string

In [ ]:
table.find("tr").find('td', class_='left', attrs={'data-stat': 'match_report'}).find('a').get('href')

In [ ]:
# target is to get home team name
for game in table.find_all("tr"):
    ht = game.find('td', class_='right', attrs={'data-stat': 'home_team'}).string
    print(ht)

In [ ]:
# target is to get home team link
for game in table.find_all("tr"):
    if "spacer" in game.get('class', []) and "partial_table" in game.get('class', []) and "result_all" in game.get('class', []):
        continue
    ht = game.find('td', class_='right', attrs={'data-stat': 'home_team'}).find('a').get('href')
    print(ht)

In [ ]:
squad_id_pattern = r"/squads/([a-f0-9]+)/"

test_team = '/en/squads/1df6b87e/2023-2024/Sheffield-United-Stats'
ht = re.search(squad_id_pattern, test_team)

In [ ]:
ht.match()

In [ ]:
# target is to get the report match link
i=0
for game in table.find_all("tr"):
    if "spacer" in game.get('class', []) and "partial_table" in game.get('class', []) and "result_all" in game.get('class', []):
        continue
    mr = game.find('td', class_='left', attrs={'data-stat': 'match_report'}).find('a').get('href')
    i+=1 
    print(i, mr)

In [128]:
test = '/en/matches/3a6836b4/Burnley-Manchester-City-August-11-2023-Premier-League'

In [129]:
test.split('/')[3]

'3a6836b4'

In [130]:
match = re.search(game_id_pattern, test)

In [131]:
match

<re.Match object; span=(3, 21), match='/matches/3a6836b4/'>

## Get proper table

In [132]:
fixtures = df[0].dropna(how='all')
fixtures.head(5)

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City,21572.0,Turf Moor,Craig Pawson,Match Report,NaN
1,1.0,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,Emirates Stadium,Michael Oliver,Match Report,NaN
2,1.0,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham,39940.0,Goodison Park,Stuart Attwell,Match Report,NaN
3,1.0,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,Bramall Lane,John Brooks,Match Report,NaN
4,1.0,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town,31872.0,The American Express Community Stadium,David Coote,Match Report,NaN


In [ ]:
fixtures.info()

In [133]:
fixtures = fixtures[['Wk', 'Date', 'Home', 'Score', 'Away']]
fixtures.rename(columns={'Wk': 'gameweek',
                         'Date': 'date',
                         'Home': 'home_team',
                         'Score': 'score',
                         'Away': 'away_team'}, inplace=True)

fixtures['home_team'] = fixtures['home_team'].astype(str)
fixtures['away_team'] = fixtures['away_team'].astype(str)
fixtures['gameweek'] = fixtures['gameweek'].astype(int)
fixtures['date'] = pd.to_datetime(fixtures['date'])
        
fixtures.head(10)

,gameweek,date,home_team,score,away_team
0,1,2023-08-11,Burnley,0–3,Manchester City
1,1,2023-08-12,Arsenal,2–1,Nott'ham Forest
2,1,2023-08-12,Everton,0–1,Fulham
3,1,2023-08-12,Sheffield Utd,0–1,Crystal Palace
4,1,2023-08-12,Brighton,4–1,Luton Town
5,1,2023-08-12,Bournemouth,1–1,West Ham
6,1,2023-08-12,Newcastle Utd,5–1,Aston Villa
7,1,2023-08-13,Brentford,2–2,Tottenham
8,1,2023-08-13,Chelsea,1–1,Liverpool
9,1,2023-08-14,Manchester Utd,1–0,Wolves


In [134]:
# Get the html pf the fictures table
table = soup.find(id='sched_2023-2024_9_1').find("tbody")
game_ids = []
home_ids = []
away_ids = []
squad_id_pattern = r"/squads/([a-f0-9]+)/"
game_id_pattern = r"/matches/([a-f0-9]+)/"

# Loop through the rows of the table
for game in table.find_all("tr"):
    # Skip rows with spacer
    if "spacer" in game.get('class', []) and "partial_table" in game.get('class', []) and "result_all" in game.get('class', []):
        continue

    # Extract match report / home / away URLs
    match_report = game.find('td', class_='left', attrs={'data-stat': 'match_report'})
    home_team = game.find('td', class_='right', attrs={'data-stat': 'home_team'})
    away_team = game.find('td', class_='left', attrs={'data-stat': 'away_team'})
    
    # Check if the match_report element is found
    if match_report:
        mr = match_report.find('a').get('href')
        ht = home_team.find('a').get('href')
        at = away_team.find('a').get('href')
        # extract game & teams ids using regex
        game_id = re.search(game_id_pattern, mr)
        home_id = re.search(squad_id_pattern, ht)
        away_id = re.search(squad_id_pattern, at)
        # Store ids
        game_ids.append(game_id.group(1) if game_id else '')
        home_ids.append(home_id.group(1) if home_id else '')
        away_ids.append(away_id.group(1) if away_id else '')
        

# Create a DataFrame from the list of match id
df_id = pd.DataFrame({
                'game_id': game_ids, 
                'home_id': home_ids,
                'away_id': away_ids})

df_id.head(3)

,game_id,home_id,away_id
0,3a6836b4,943e8050,b8fd03ef
1,26a7f90c,18bb7c10,e4a775cb
2,15addfc7,d3fd31cc,fd962109


In [135]:
df_games = pd.concat([fixtures, df_id], axis=1)
df_games.head(4)

,gameweek,date,home_team,score,away_team,game_id,home_id,away_id
0,1.0,2023-08-11,Burnley,0–3,Manchester City,3a6836b4,943e8050,b8fd03ef
1,1.0,2023-08-12,Arsenal,2–1,Nott'ham Forest,26a7f90c,18bb7c10,e4a775cb
2,1.0,2023-08-12,Everton,0–1,Fulham,15addfc7,d3fd31cc,fd962109
3,1.0,2023-08-12,Sheffield Utd,0–1,Crystal Palace,55fd92c7,1df6b87e,47c64c55


In [136]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 0 to 373
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   gameweek   380 non-null    float64       
 1   date       380 non-null    datetime64[ns]
 2   home_team  380 non-null    object        
 3   score      380 non-null    object        
 4   away_team  380 non-null    object        
 5   game_id    380 non-null    object        
 6   home_id    380 non-null    object        
 7   away_id    380 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 29.4+ KB


In [143]:
import subprocess
import os

# Get the root directory of the git repository
def get_git_root():
    return subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')

# Use the function
repo_root = get_git_root()

repo_root

'/Users/alexisfayolle/code/alexisfyll/bookmaker'

In [144]:
df_games.to_csv(f'{repo_root}/raw_data/data_pl_23-24.csv')